# 🌍 MASt3R-SLAM + Stella World Builder

This notebook processes a video using MASt3R-SLAM to create:
1. **3D Point Cloud** (PLY file)
2. **.stella Explorable World** (ZIP container with collision and render mesh)

## Instructions:
1. Click **Runtime → Change runtime type** and select **GPU**
2. Upload your video when prompted
3. Run all cells (Runtime → Run all)
4. Download the outputs at the end

**Estimated time:** 15-30 minutes for a 2-3 minute video

## Step 1: Setup MASt3R-SLAM

In [ ]:
# Check GPU
!nvidia-smi
print("\n✅ GPU detected! Ready to process video.")

In [ ]:
# Clone MASt3R-SLAM
!git clone https://github.com/rmurai0610/MASt3R-SLAM.git --recursive
%cd MASt3R-SLAM

In [ ]:
# Install dependencies with numpy compatibility fix
print("🔧 Installing PyTorch...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

print("🔧 Installing MASt3R (this may show numpy warnings - they're OK)...")
# Install with --no-deps first, then fix numpy
!pip install -q --no-deps -e thirdparty/mast3r
!pip install -q --no-deps -e thirdparty/in3d

print("🔧 Installing core dependencies...")
# Install compatible numpy version for MASt3R
!pip install -q "numpy>=1.26,<2.0" --force-reinstall

print("🔧 Installing remaining packages...")
!pip install -q --no-build-isolation -e .
!pip install -q trimesh scipy plyfile

# Reinstall opencv to match numpy 1.26
!pip uninstall -y opencv-python opencv-contrib-python opencv-python-headless 2>/dev/null || true
!pip install -q opencv-python==4.9.0.80

print("\n✅ Dependencies installed (numpy compatibility fixed)")

In [ ]:
# Download model checkpoints (~1.5GB)
import os
os.makedirs('checkpoints', exist_ok=True)

!wget -q --show-progress https://download.europe.naverlabs.com/ComputerVision/MASt3R/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric.pth -P checkpoints/
!wget -q --show-progress https://download.europe.naverlabs.com/ComputerVision/MASt3R/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric_retrieval_trainingfree.pth -P checkpoints/
!wget -q --show-progress https://download.europe.naverlabs.com/ComputerVision/MASt3R/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric_retrieval_codebook.pkl -P checkpoints/

print("\n✅ Model checkpoints downloaded")

## Step 2: Upload Your Video

In [ ]:
from google.colab import files
import os

print("📹 Upload your video file (MP4, MOV, AVI)")
uploaded = files.upload()

video_filename = list(uploaded.keys())[0]
video_name = os.path.splitext(video_filename)[0]

print(f"\n✅ Video uploaded: {video_filename}")
print(f"   Output name: {video_name}")

## Step 3: Run MASt3R-SLAM (This takes 15-30 minutes)

In [ ]:
import time
start_time = time.time()

print("🚀 Running MASt3R-SLAM...")
print("This will take 15-30 minutes depending on video length.")
print("\n" + "="*60 + "\n")

!python main.py \
    --dataset "{video_filename}" \
    --save-as "{video_name}" \
    --config config/base.yaml \
    --no-viz

elapsed = time.time() - start_time
print(f"\n✅ MASt3R-SLAM completed in {elapsed/60:.1f} minutes")
print(f"   Point cloud saved to: logs/{video_name}/{video_name}.ply")

## Step 4: Install Stella Package

In [ ]:
# Clone and install stella package
!git clone https://github.com/ZRosserMcIntosh/mast3r-slam-stella.git || echo "Using local version"

# Create stella package inline if clone fails
import os
os.makedirs('stella_pkg', exist_ok=True)

# We'll create a minimal version for Colab
print("📦 Setting up Stella package...")
!pip install -q trimesh
print("✅ Ready to create .stella files")

## Step 5: Create .stella World File

In [ ]:
# Inline stella builder for Colab
import numpy as np
import trimesh
import zipfile
import json
import struct
from pathlib import Path

def load_ply(ply_path):
    """Load point cloud from PLY"""
    mesh = trimesh.load(ply_path)
    points = np.array(mesh.vertices)
    colors = None
    if hasattr(mesh, 'visual') and hasattr(mesh.visual, 'vertex_colors'):
        colors = np.array(mesh.visual.vertex_colors)[:, :3]
    return points, colors

def voxelize_simple(points, voxel_size=0.1):
    """Simple voxelization"""
    min_bound = points.min(axis=0)
    max_bound = points.max(axis=0)
    dims = np.ceil((max_bound - min_bound) / voxel_size).astype(int) + 1
    
    voxel_coords = np.floor((points - min_bound) / voxel_size).astype(int)
    voxel_coords = np.clip(voxel_coords, 0, dims - 1)
    
    grid = np.zeros(dims, dtype=bool)
    grid[voxel_coords[:, 0], voxel_coords[:, 1], voxel_coords[:, 2]] = True
    
    return grid, min_bound, voxel_size

def write_rlevox_simple(path, grid, voxel_size, origin):
    """Write RLEVOX collision file"""
    with open(path, 'wb') as f:
        # Header
        f.write(b'STVX')  # Magic
        f.write(struct.pack('<I', 1))  # Version
        f.write(struct.pack('<III', *grid.shape))  # Dimensions
        f.write(struct.pack('<f', voxel_size))  # Voxel size
        f.write(struct.pack('<fff', *origin))  # Origin
        
        # RLE encode
        flat = grid.astype(np.uint8).flatten()
        rle_data = []
        i = 0
        while i < len(flat):
            val = flat[i]
            count = 1
            while i + count < len(flat) and flat[i + count] == val and count < 255:
                count += 1
            rle_data.append(bytes([val, count]))
            i += count
        
        payload = b''.join(rle_data)
        f.write(struct.pack('<I', len(payload)))  # Payload size
        f.write(b'\x00' * 28)  # Reserved
        f.write(payload)

def create_point_mesh(points, colors, max_points=50000):
    """Create render mesh from points"""
    if len(points) > max_points:
        indices = np.random.choice(len(points), max_points, replace=False)
        points = points[indices]
        if colors is not None:
            colors = colors[indices]
    
    sphere = trimesh.creation.icosphere(subdivisions=0, radius=0.01)
    meshes = []
    
    for i, pt in enumerate(points[:5000]):  # Limit for Colab
        s = sphere.copy()
        s.apply_translation(pt)
        if colors is not None and i < len(colors):
            color = colors[i] if colors[i].max() > 1 else (colors[i] * 255).astype(np.uint8)
            s.visual.vertex_colors = np.tile(np.append(color, 255), (len(s.vertices), 1))
        meshes.append(s)
    
    return trimesh.util.concatenate(meshes) if meshes else trimesh.Trimesh()

def create_stella(ply_path, output_path, title="3D World"):
    """Create .stella file from PLY"""
    print(f"Loading PLY: {ply_path}")
    points, colors = load_ply(ply_path)
    print(f"Loaded {len(points)} points")
    
    # Align to floor
    floor_y = np.percentile(points[:, 1], 5)
    points[:, 1] -= floor_y
    
    # Voxelize
    print("Voxelizing...")
    grid, origin, voxel_size = voxelize_simple(points, voxel_size=0.1)
    print(f"Grid: {grid.shape}, {grid.sum()} solid voxels")
    
    # Create mesh
    print("Creating render mesh...")
    mesh = create_point_mesh(points, colors)
    
    # Create manifest
    manifest = {
        "schema": "https://virgil.systems/schemas/stella/manifest/v1.schema.json",
        "name": title,
        "version": "1.0.0",
        "levels": [{"id": "0", "name": "Main"}]
    }
    
    level_json = {
        "schema": "https://virgil.systems/schemas/stella/level/v1.schema.json",
        "name": "Main Level",
        "spawn": {
            "position": [float(origin[0]), 1.7, float(origin[2])],
            "yaw_degrees": 0.0
        },
        "render": {"uri": "render.glb"},
        "collision": {
            "uri": "collision.rlevox",
            "player": {"height_m": 1.7, "radius_m": 0.3}
        }
    }
    
    # Write files
    import tempfile
    with tempfile.TemporaryDirectory() as tmpdir:
        tmpdir = Path(tmpdir)
        
        collision_path = tmpdir / "collision.rlevox"
        write_rlevox_simple(collision_path, grid, voxel_size, origin)
        
        render_path = tmpdir / "render.glb"
        mesh.export(str(render_path))
        
        # Create ZIP
        with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zf:
            zf.writestr('manifest.json', json.dumps(manifest, indent=2))
            zf.writestr('levels/0/level.json', json.dumps(level_json, indent=2))
            zf.write(render_path, 'levels/0/render.glb')
            zf.write(collision_path, 'levels/0/collision.rlevox')
    
    print(f"✅ Created: {output_path}")

# Run it
ply_file = f"logs/{video_name}/{video_name}.ply"
stella_file = f"{video_name}.stella"

print("🌍 Creating .stella world...")
create_stella(ply_file, stella_file, title=video_name.replace('_', ' ').title())

print(f"\n✅ DONE!")
print(f"   PLY:    {ply_file}")
print(f"   Stella: {stella_file}")

## Step 6: Download Results

In [ ]:
from google.colab import files
import os

print("📥 Downloading files...\n")

# Download PLY
ply_path = f"logs/{video_name}/{video_name}.ply"
if os.path.exists(ply_path):
    print(f"Downloading PLY: {ply_path}")
    files.download(ply_path)
else:
    print(f"⚠️ PLY not found: {ply_path}")

# Download .stella
stella_path = f"{video_name}.stella"
if os.path.exists(stella_path):
    print(f"Downloading .stella: {stella_path}")
    files.download(stella_path)
else:
    print(f"⚠️ .stella not found: {stella_path}")

print("\n✅ All done! Check your Downloads folder.")

## 🎉 You're Done!

### What you got:
1. **`.ply` file** - 3D point cloud, open in MeshLab/CloudCompare/Blender
2. **`.stella` file** - Explorable 3D world with collision detection

### Next steps:
- Open the `.stella` file in VS Code with the extension installed
- Or extract it: `stella extract apartment.stella ./output/`
- Use WASD + mouse to explore in 3D!